<a href="https://colab.research.google.com/github/tajski/Projekt-Sieci-Neuronowe/blob/main/Projekt_Sieci_Neuronowe_BT_KW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
from matplotlib import pyplot as plt
from keras.utils import image_dataset_from_directory
from keras import layers, Model
from keras.models import Sequential
from keras.utils import plot_model
from keras.metrics import CategoricalAccuracy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives, Accuracy

In [ ]:
drive.mount('/content/drive', force_remount=True)
!unzip -q "/content/drive/MyDrive/Colab Notebooks/diabetic.zip"

Mounted at /content/drive


In [ ]:
train_data_root = "train"
test_data_root = "test"
valid_data_root = "valid"

train_data = {}
test_data = {}
valid_data = {}

# Przechodzenie przez katalogi treningowe
for root, dirs, files in os.walk(train_data_root):
    for dir in dirs:
        dir_path = os.path.join(root, dir)
        train_data[dir] = os.listdir(dir_path)

print("Dane treningowe:")
for x in train_data:
    print(x, len(train_data[x]))

# Przechodzenie przez katalogi testowe
for root, dirs, files in os.walk(test_data_root):
    for dir in dirs:
        dir_path = os.path.join(root, dir)
        test_data[dir] = os.listdir(dir_path)

print()

print("Dane testowe:")
for x in test_data:
    print(x, len(test_data[x]))

    # Przechodzenie przez katalogi walidacyjne
for root, dirs, files in os.walk(valid_data_root):
    for dir in dirs:
        dir_path = os.path.join(root, dir)
        valid_data[dir] = os.listdir(dir_path)

print()

print("Dane treningowe:")
for x in valid_data:
    print(x, len(valid_data[x]))

Dane treningowe:
DR 1050
No_DR 1026

Dane testowe:
DR 113
No_DR 118

Dane treningowe:
DR 245
No_DR 286
